In [2]:
import pandas as pd
import numpy as np
import os, sys

In [3]:
# GLOBALS
LOCAL_ROOT = '/Users/varunn/Documents/ExternalTest/'
PROJ_DIR = os.path.join(LOCAL_ROOT, 'Udaan')
INP_FN = os.path.join(PROJ_DIR, 'underwriting_problem_with_predictor.xlsx')
SHEET_NAME = 'raw data'

In [4]:
assert os.path.isfile(INP_FN)

In [5]:
# read data
df = pd.read_excel(INP_FN, sheet_name=SHEET_NAME)

In [51]:
print(df.shape)
df.head()

(51946, 43)


,id,category,activation_type,business_category,head_office_pincode,tags_bucket,orders_delivered_before_credit_activation,rto_orders_before_credit_activation,gmv_delivered_before_credit_activation,no_months_orders_delivered_before_credit_activation,...,distinct_device_brand_before_credit_activation,distinct_device_model_before_credit_activation,session_count_before_credit_activation,last_30_days_session_count_before_credit_activation,last_7_days_session_count_before_credit_activation,session_count_before_credit_activation_0_5,session_count_before_credit_activation_6_11,session_count_before_credit_activation_12_17,session_count_before_credit_activation_18_23,binary_predictor
0,1002,A,N,retail,NaN,HEAVY,0.0,0.0,NaN,0.0,...,3.0,3.0,233.0,42.0,12.0,15.0,31.0,40.0,147.0,0
1,1003,A,N,retail,577002.0,HEAVY,NaN,NaN,NaN,NaN,...,2.0,2.0,166.0,120.0,11.0,1.0,9.0,82.0,74.0,0
2,1010,A,N,retail,NaN,HEAVY,9.0,3.0,28632.75,1.0,...,2.0,3.0,95.0,85.0,85.0,1.0,6.0,59.0,29.0,1
3,1011,A,N,retail,NaN,HEAVY,NaN,NaN,NaN,NaN,...,1.0,1.0,20.0,1.0,0.0,0.0,2.0,18.0,0.0,1
4,1012,A,N,retail,NaN,HEAVY,NaN,NaN,NaN,NaN,...,1.0,1.0,71.0,25.0,8.0,NaN,8.0,23.0,40.0,1


In [7]:
# GLOBAL VARIABLE NAMES
DV = 'binary_predictor'
ID = 'id'
SAMPLE = 'sample'
DISCRETE_FEATS = ['category', 'activation_type', 'business_category',
                 'head_office_pincode', 'tags_bucket']
IGNORE_FEATS = ['first_order_reg_duration']
SCORE_FEATS = ['cibil_score']
NUMERIC_FEATS = [x for x in list(df.columns) if x not in
                 [DV, ID]+DISCRETE_FEATS+IGNORE_FEATS+SCORE_FEATS]
FEATS = DISCRETE_FEATS + NUMERIC_FEATS + SCORE_FEATS

In [8]:
print(len(FEATS), '\t', len(DISCRETE_FEATS), '\t', len(SCORE_FEATS), '\t', len(NUMERIC_FEATS))

(40, '\t', 5, '\t', 1, '\t', 34)


In [9]:
print(df[DV].value_counts())
print(df[DV].mean())

0    26623
1    25323
Name: binary_predictor, dtype: int64
0.4874870057367266


In [10]:
# Split data into train and test
from sklearn.model_selection import train_test_split


def sample_split(data, split=0.3, random_state=121212, feats=FEATS,
                 dv=DV, idx=ID):
    """
    stratified sampling into 70% dev and 30% val
    """
    df = data.copy()
    x_train, x_test, y_train, y_test = train_test_split(
     df[feats+[idx]], df[dv], test_size=split,
     random_state=random_state)
    # dev and val samples
    dev = pd.concat([x_train, y_train], axis=1)
    dev['sample'] = 'dev'
    val = pd.concat([x_test, y_test], axis=1)
    val['sample'] = 'val'
    cols = feats + [idx, dv, 'sample']
    df1 = pd.concat([dev[cols], val[cols]], axis=0)
    df1.reset_index(drop=True, inplace=True)
    return df1

In [11]:
df1 = sample_split(df)    # 70% train and 30% test

In [12]:
print(df1.shape)
print(df1[SAMPLE].value_counts())
print(pd.crosstab(df1[SAMPLE], df1[DV]))
print(df1.groupby(SAMPLE)[DV].mean())

(51946, 43)
dev    36362
val    15584
Name: sample, dtype: int64
binary_predictor      0      1
sample                        
dev               18650  17712
val                7973   7611
sample
dev    0.487102
val    0.488386
Name: binary_predictor, dtype: float64


In [13]:
df[FEATS].min()

category                                                       A
activation_type                                                C
business_category                                         retail
head_office_pincode                                         1123
tags_bucket                                                HEAVY
orders_delivered_before_credit_activation                      0
rto_orders_before_credit_activation                            0
gmv_delivered_before_credit_activation                      33.6
no_months_orders_delivered_before_credit_activation            0
last_30_days_orders_delivered_before_credit_activation         0
last_30_days_gmv_delivered_before_credit_activation        31.38
no_months_transaction_done_before_credit_activation            1
total_orders_before_credit_activation                          0
last_30_days_rto_orders_before_credit_activation               0
no_weeks_orders_delivered_before_credit_activation             0
no_weeks_transaction_done

In [14]:
# Handling discrete features

In [15]:
# function to group values of discrete variables to account for low frequencies
from math import log


def group_cats_with_lower_freq(data, cat_col, freq_thresh=20):
    """
    groups values which are of lower frequencies
    """
    mask = data[SAMPLE] == 'dev'
    cats = data.loc[mask, cat_col].value_counts().to_dict()
    good_cats = [k for k, v in cats.items() if v >= freq_thresh]
    return good_cats


def get_woe(feat_series, dv_series):
    """
    returns WOE which stands for weight of evidence
    """
    table = pd.crosstab(feat_series, dv_series)
    table.reset_index(inplace=True)
    table.rename(columns={0: '#good', 1: '#bad'}, inplace=True)
    total_good = table['#good'].sum()
    total_bad = table['#bad'].sum()
    table['perc_good'] = table['#good'].apply(lambda x: x/float(total_good))
    table['perc_bad'] = table['#bad'].apply(lambda x: x/float(total_bad))
    mask = (table['perc_good'] != 0) & (table['perc_bad'] != 0)
    table.loc[mask, 'WOE'] = list(map(
     lambda x, y: log(x / float(y)), table.loc[mask, 'perc_good'],
     table.loc[mask, 'perc_bad']))
    table.loc[~mask, 'WOE'] = np.NaN
    table.reset_index(drop=True, inplace=True)
    return table

In [16]:
for feat in DISCRETE_FEATS:
    print(feat)
    mask = df1[SAMPLE] == 'dev'
    print(df1.loc[mask, feat].nunique(), '\n')

category
(8, '\n')
activation_type
(3, '\n')
business_category
(2, '\n')
head_office_pincode
(3604, '\n')
tags_bucket
(2, '\n')


In [17]:
# group head_office_pincode
mask = df1[SAMPLE] == 'dev'
MIN_FREQ = round(float(0.01*mask.sum()))
print(MIN_FREQ)

for col in ['head_office_pincode']:
    print(col)
    # grouping values based on freq
    grouped_values = group_cats_with_lower_freq(df1, col,
                                                freq_thresh=MIN_FREQ)
    mask = df1[col].isin(grouped_values)
    # imputing values with freq < 10000
    df1.loc[~mask, col] = 'others'

364.0
head_office_pincode


In [18]:
print(df1['head_office_pincode'].value_counts())

others      50608
122001.0      758
560068.0      580
Name: head_office_pincode, dtype: int64


In [19]:
# WOE mapping for discrete vars
for feat in DISCRETE_FEATS:
    print(feat)
    mask = df1[SAMPLE] == 'dev'
    table = get_woe(df1.loc[mask, feat], df1.loc[mask, DV])
    map_dict = dict(zip(table[feat], table['WOE']))
    # encoding
    df1[feat] = df1[feat].map(map_dict)

category
activation_type
business_category
head_office_pincode
tags_bucket


In [20]:
df1['tags_bucket'].describe()

count    51946.000000
mean        -0.000079
std          0.029331
min         -0.144237
25%          0.005889
50%          0.005889
75%          0.005889
max          0.005889
Name: tags_bucket, dtype: float64

In [21]:
df1[FEATS].dtypes

category                                                  float64
activation_type                                           float64
business_category                                         float64
head_office_pincode                                       float64
tags_bucket                                               float64
orders_delivered_before_credit_activation                 float64
rto_orders_before_credit_activation                       float64
gmv_delivered_before_credit_activation                    float64
no_months_orders_delivered_before_credit_activation       float64
last_30_days_orders_delivered_before_credit_activation    float64
last_30_days_gmv_delivered_before_credit_activation       float64
no_months_transaction_done_before_credit_activation       float64
total_orders_before_credit_activation                     float64
last_30_days_rto_orders_before_credit_activation          float64
no_weeks_orders_delivered_before_credit_activation        float64
no_weeks_t

In [33]:
mask = df1[SAMPLE] == 'dev'
df1.loc[mask, 'cibil_score'].median()

715.0

In [23]:
# Cibil score can't be below 500
mask = df1['cibil_score'] < 500
df1.loc[mask, 'cibil_score'] = None

In [24]:
# summary stats for features in dev sample

from sklearn.metrics import roc_auc_score

def get_summary_stats(data, feats=FEATS):
    
    data1 = data.copy()
    mask = data1[SAMPLE] == 'dev'
    df = data1.loc[mask, :]
    df.reset_index(drop=True, inplace=True)
    summary_df = pd.DataFrame({'feature': feats,
                               'dtype': df[feats].dtypes})
    
    summary_df['_min'] = df[feats].min()
    summary_df['_max'] = df[feats].max()
    summary_df['_mean'] = df[feats].mean()
    summary_df['_median'] = df[feats].median()
    summary_df['_std'] = df[feats].std()
    summary_df['num_missing'] = df[feats].isnull().sum()
    summary_df['perc_missing'] = summary_df['num_missing'].apply(
        lambda x: 1.*x/df.shape[0])
    aucs = []    # median imputation
    for feat in feats:
        print(feat)
        _median = df[feat].median()
        # impute
        mask = df[feat].isnull()
        df.loc[mask, feat] = _median
        _auc = roc_auc_score(df[DV], df[feat])
        aucs.append(_auc)
    summary_df['auc'] = aucs
    
    return summary_df

In [25]:
summary_stats_df = get_summary_stats(df1)

category
activation_type


/Users/varunn/.virtualenvs/xgb/lib/python2.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


business_category
head_office_pincode
tags_bucket
orders_delivered_before_credit_activation
rto_orders_before_credit_activation
gmv_delivered_before_credit_activation
no_months_orders_delivered_before_credit_activation
last_30_days_orders_delivered_before_credit_activation
last_30_days_gmv_delivered_before_credit_activation
no_months_transaction_done_before_credit_activation
total_orders_before_credit_activation
last_30_days_rto_orders_before_credit_activation
no_weeks_orders_delivered_before_credit_activation
no_weeks_transaction_done_before_credit_activation
last_30_days_transaction_done_before_credit_activation
last_7_days_transaction_done_before_credit_activation
last_7_days_orders_delivered_before_credit_activation
last_7_days_gmv_delivered_before_credit_activation
total_gmv_before_credit_activation
last_7_days_rto_orders_before_credit_activation
last_30_days_gmv_before_credit_activation
last_7_days_gmv_before_credit_activation
num_seller_per_buyer_before_credit_activation
num_mob

In [26]:
print(summary_stats_df.shape)
print(summary_stats_df.head())

(40, 10)
                       dtype              feature      _min      _max  \
category             float64             category -0.370855  1.481294   
activation_type      float64      activation_type -0.243549  0.972901   
business_category    float64    business_category -0.178088  0.000935   
head_office_pincode  float64  head_office_pincode -0.002706  0.150991   
tags_bucket          float64          tags_bucket -0.144237  0.005889   

                            _mean   _median      _std  num_missing  \
category             4.663790e-03  0.166696  0.362782            1   
activation_type      3.244977e-04  0.155637  0.198310            0   
business_category   -3.092270e-07  0.000935  0.012907            0   
head_office_pincode  8.758570e-06 -0.002706  0.018856            0   
tags_bucket          1.166146e-06  0.005889  0.029141            0   

                     perc_missing       auc  
category                 0.000028  0.412032  
activation_type          0.000000  0.45

In [27]:
print(summary_stats_df[summary_stats_df['auc'] > 0.5])

                                                    dtype  \
rto_orders_before_credit_activation               float64   
last_30_days_rto_orders_before_credit_activation  float64   
last_7_days_rto_orders_before_credit_activation   float64   
last_30_days_gmv_before_credit_activation         float64   
last_7_days_gmv_before_credit_activation          float64   
distinct_device_brand_before_credit_activation    float64   
distinct_device_model_before_credit_activation    float64   
session_count_before_credit_activation_6_11       float64   

                                                                                           feature  \
rto_orders_before_credit_activation                            rto_orders_before_credit_activation   
last_30_days_rto_orders_before_credit_activation  last_30_days_rto_orders_before_credit_activation   
last_7_days_rto_orders_before_credit_activation    last_7_days_rto_orders_before_credit_activation   
last_30_days_gmv_before_credit_activation 

In [28]:
print(summary_stats_df['auc'].describe())

count    40.000000
mean      0.485505
std       0.023638
min       0.412032
25%       0.474110
50%       0.490431
75%       0.499542
max       0.530230
Name: auc, dtype: float64


In [29]:
summary_stats_df[summary_stats_df['feature'] == 'cibil_score']

,dtype,feature,_min,_max,_mean,_median,_std,num_missing,perc_missing,auc
cibil_score,float64,cibil_score,518.0,887.0,708.755263,715.0,54.013862,35602,0.979099,0.498


In [30]:
# save
summary_stats_df.to_csv(os.path.join(PROJ_DIR, 'summary_stats_dev.csv'),
                        index=False)

In [31]:
# Preprocessing

In [34]:
medians = []
for feat in FEATS:
    print(feat)
    mask1 = df1[SAMPLE] == 'dev'
    _median = df1.loc[mask, feat].median()
    print(_median)
    medians.append(_median)

category
0.1666963190322269
activation_type
0.1556373053966074
business_category
0.0009351282804697648
head_office_pincode
-0.002706144402511287
tags_bucket
0.005888589286381207
orders_delivered_before_credit_activation
7.0
rto_orders_before_credit_activation
0.0
gmv_delivered_before_credit_activation
14757.37
no_months_orders_delivered_before_credit_activation
2.0
last_30_days_orders_delivered_before_credit_activation
2.0
last_30_days_gmv_delivered_before_credit_activation
8402.0
no_months_transaction_done_before_credit_activation
2.0
total_orders_before_credit_activation
9.0
last_30_days_rto_orders_before_credit_activation
0.0
no_weeks_orders_delivered_before_credit_activation
4.0
no_weeks_transaction_done_before_credit_activation
4.0
last_30_days_transaction_done_before_credit_activation
3.0
last_7_days_transaction_done_before_credit_activation
0.0
last_7_days_orders_delivered_before_credit_activation
0.0
last_7_days_gmv_delivered_before_credit_activation
4446.3
total_gmv_before_cre

In [35]:
impute_dct = dict(zip(FEATS, medians))
df1_pre = df1.copy()
df1_pre.fillna(impute_dct, inplace=True)

In [36]:
impute_dct['cibil_score']

715.0

In [37]:
print(df1_pre.shape)
print(df1_pre[FEATS].isnull().sum())

(51946, 43)
category                                                  0
activation_type                                           0
business_category                                         0
head_office_pincode                                       0
tags_bucket                                               0
orders_delivered_before_credit_activation                 0
rto_orders_before_credit_activation                       0
gmv_delivered_before_credit_activation                    0
no_months_orders_delivered_before_credit_activation       0
last_30_days_orders_delivered_before_credit_activation    0
last_30_days_gmv_delivered_before_credit_activation       0
no_months_transaction_done_before_credit_activation       0
total_orders_before_credit_activation                     0
last_30_days_rto_orders_before_credit_activation          0
no_weeks_orders_delivered_before_credit_activation        0
no_weeks_transaction_done_before_credit_activation        0
last_30_days_transaction_don

In [38]:
# getting train and test data
mask = df1_pre[SAMPLE] == "dev"
x_pre_train, y_pre_train = df1_pre.loc[mask, FEATS], df1_pre.loc[mask, DV]
x_pre_test, y_pre_test = df1_pre.loc[~mask, FEATS], df1_pre.loc[~mask, DV]

In [42]:
# Modelling

# LR (l1, l2, EN)

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000, C=1.5,
                         penalty='l1')
%time clf.fit(x_pre_train, y_pre_train)

CPU times: user 14 s, sys: 70.3 ms, total: 14.1 s
Wall time: 14.1 s


LogisticRegression(C=1.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [43]:
# prediction
def get_prediction(model, x_test):
    return model.predict_proba(x_test)[:, 1]

def auc(actual, pred_prob):
    return roc_auc_score(actual, pred_prob)

In [44]:
# LR
print("test AUC: %0.4f" %auc(y_pre_test, get_prediction(clf, x_pre_test)))
print("train AUC: %0.4f" %auc(y_pre_train, get_prediction(clf, x_pre_train)))

test AUC: 0.6462
train AUC: 0.6512


In [45]:
# XGBoost

import xgboost as xgb
import operator

MODEL_PATH = os.path.join(PROJ_DIR, 'models')
if not os.path.isdir(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
    
def create_feature_map(features, fn):
    outfile = open(os.path.join(MODEL_PATH,fn), 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()

def runXGB(fn, train_X, train_y, test_X, test_y=None, test_X2=None,
           feature_names=None, seed_val=0, rounds=500, dep=8, eta=0.05):
    params = {}
    params["objective"] = "binary:logistic"
    params['eval_metric'] = 'auc'
    params["eta"] = eta
    params["subsample"] = 0.8
    params["min_child_weight"] = 1
    params["colsample_bytree"] = 0.7
    params["max_depth"] = dep

    params["silent"] = 1
    params["seed"] = seed_val
    # params["max_delta_step"] = 2
    # params["gamma"] = 0.5
    num_rounds = rounds

    plst = list(params.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [(xgtrain, 'train'), (xgtest, 'test')]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist,
                          early_stopping_rounds=20, verbose_eval=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    if feature_names is not None:
        create_feature_map(feature_names, fn)
        model.dump_model(os.path.join(MODEL_PATH,'raw_feats_xgb.txt'),
                         os.path.join(MODEL_PATH,fn), with_stats=True)
        importance = model.get_fscore(fmap=os.path.join(MODEL_PATH,fn))
        importance = sorted(importance.items(), key=operator.itemgetter(1),
                            reverse=True)
        imp_df = pd.DataFrame(importance, columns=['feature', 'fscore'])
        imp_df['fscore'] = imp_df['fscore'] / imp_df['fscore'].sum()
        imp_df.to_csv(os.path.join(MODEL_PATH,"raw_imp_feats_xgb.csv"),
                      index=False)

    pred_test_y = model.predict(xgtest, ntree_limit=model.best_ntree_limit)
    if test_X2:
        pred_test_y2 = model.predict(xgb.DMatrix(test_X2),
                                     ntree_limit=model.best_ntree_limit)
    else:
        pred_test_y2 = None

    loss = 0
    if test_y is not None:
        loss = metrics.roc_auc_score(test_y, pred_test_y)

    return pred_test_y, loss, pred_test_y2, model

In [46]:
# XGB algo can handle missing values. So, imputation is not needed
# train and test data on raw data
mask = df1[SAMPLE] == "dev"
x_train_raw, y_train_raw = df1.loc[mask, FEATS], df1.loc[mask, DV]
dev_cids = df1.loc[mask, ID].tolist()
x_test_raw, y_test_raw = df1.loc[~mask, FEATS], df1.loc[~mask, DV]
val_cids = df1.loc[~mask, ID].tolist()

In [47]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test_raw.shape)
print(y_test_raw.shape)
print(y_train_raw.value_counts())
print(y_test_raw.value_counts())

(36362, 40)
(36362,)
(15584, 40)
(15584,)
0    18650
1    17712
Name: binary_predictor, dtype: int64
0    7973
1    7611
Name: binary_predictor, dtype: int64


In [50]:
# XGB model with transaction features excluding count based features
import time
from sklearn import metrics
start = time.time()
pred_val, loss, pred_test, model = runXGB(
                 "raw_feats_xgb.fmap", x_train_raw,
                 y_train_raw, x_test_raw, y_test_raw,
                 None, rounds=200, dep=4, feature_names=FEATS)
print("time taken: %0.2f" %(time.time()-start))

[0]	train-auc:0.637233	test-auc:0.62286
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 20 rounds.
[20]	train-auc:0.665731	test-auc:0.644852
[40]	train-auc:0.674637	test-auc:0.650499
[60]	train-auc:0.680961	test-auc:0.655303
[80]	train-auc:0.686481	test-auc:0.657415
[100]	train-auc:0.69095	test-auc:0.659482
[120]	train-auc:0.695248	test-auc:0.660452
[140]	train-auc:0.698566	test-auc:0.661029
[160]	train-auc:0.70166	test-auc:0.661561
[180]	train-auc:0.704589	test-auc:0.661803
[199]	train-auc:0.707819	test-auc:0.662148
time taken: 15.79
